In [1]:
pip install crewai langchain langchain-community faiss-cpu sentence-transformers transformers accelerate


  Using cached openai-1.83.0-py3-none-any.whl.metadata (25 kB)
Using cached openai-1.83.0-py3-none-any.whl (723 kB)
  Attempting uninstall: openai
    Found existing installation: openai 2.20.0
    Uninstalling openai-2.20.0:
      Successfully uninstalled openai-2.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.81.11 requires openai>=2.8.0, but you have openai 1.83.0 which is incompatible.


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

loader = TextLoader("kn.txt")
documents = loader.load()

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever()


/tmp/ipython-input-2595207823.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
def retrieve_docs(query):
    docs = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in docs])


In [4]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

hf_pipeline = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_new_tokens=150,
    temperature=0.0,
    do_sample=False,
    device_map="auto"
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


/tmp/ipython-input-3162009002.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [13]:
from crewai import Agent, Task, Crew
from crewai.tools import tool

@tool("Knowledge Base Search")
def retrieval_tool(query: str) -> str:
    """Search the knowledge base for relevant information."""
    return retrieve_docs(query)

research_agent = Agent(
    role="Research Specialist",
    goal="Retrieve relevant information before answering.",
    backstory="Expert in searching knowledge bases.",
    verbose=True,
    allow_delegation=False,
    tools=[retrieval_tool],
    llm=llm
)

answer_agent = Agent(
    role="Answer Generator",
    goal="Generate accurate answers using retrieved information.",
    backstory="Expert in explaining concepts clearly.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

research_task = Task(
    description="Search the knowledge base for information about: {question}",
    agent=research_agent,
    expected_output="Relevant documents"
)

answer_task = Task(
    description="Use the retrieved documents to answer: {question}",
    agent=answer_agent,
    expected_output="Final answer"
)

crew = Crew(
    agents=[research_agent, answer_agent],
    tasks=[research_task, answer_task],
    verbose=True
)


In [7]:
from crewai import LLM

llm = LLM(
    model="ollama/tinyllama"
)


In [6]:
!pip install litellm


  Using cached openai-2.20.0-py3-none-any.whl.metadata (29 kB)
Using cached openai-2.20.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: openai
    Found existing installation: openai 1.83.0
    Uninstalling openai-1.83.0:
      Successfully uninstalled openai-1.83.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 1.9.3 requires openai~=1.83.0, but you have openai 2.20.0 which is incompatible.
instructor 1.12.0 requires openai<2.0.0,>=1.70.0, but you have openai 2.20.0 which is incompatible.


In [ ]:
result = crew.kickoff(inputs={"question": "What is CrewAI?"})
print(result)
